In [55]:
import pandas as pd
import numpy as np
import pickle
import matplotlib.pyplot as mat
import seaborn as sns
train_csv = pd.read_csv("/kaggle/input/capstone-bank-lending-prediction/lending_train.csv")
train_csv = train_csv.drop(columns=["ID","race","extended_reason","revolving_balance"])
missing_percent = train_csv.isna().sum()*100/len(train_csv)
pd.set_option('max_columns', None)
#print(missing_val_df)
print(train_csv.dtypes)

# print(train_csv.employment.unique().tolist())/Users/chichi/Downloads/Bank_Lending.ipynb

requested_amnt              float64
loan_duration                object
employment                   object
employment_length            object
reason_for_loan              object
annual_income               float64
debt_to_income_ratio        float64
employment_verified          object
public_bankruptcies         float64
zipcode                      object
state                        object
home_ownership_status        object
delinquency_last_2yrs       float64
fico_score_range_low        float64
fico_score_range_high       float64
fico_inquired_last_6mths    float64
months_since_last_delinq    float64
total_revolving_limit       float64
type_of_application          object
any_tax_liens               float64
loan_paid                     int64
dtype: object


In [56]:
train = False
exploring = False
#no. of missing vals
missing_val_df = train_csv.shape[0]*[0]
missing_count = train_csv.isna().sum()
# for i in range (train_csv.shape[0]):
missing_val_df = train_csv.isna().sum()
train_csv["missing_vals"] = missing_val_df
#print(train_csv.columns)
# missing_val_df = pd.DataFrame({"Missing Count": missing_count})
#finding correlation between missing vals and loan paid
#print(np.corrcoef(np.array(missing_val_df).squeeze(axis = 1),train_csv['loan_paid'].to_numpy()))


In [57]:
#replacing missing vals
cat_cols = ['employment_length','zipcode','state']
num_cols = ['debt_to_income_ratio','public_bankruptcies','fico_inquired_last_6mths','fico_score_range_low','fico_score_range_high','months_since_last_delinq','total_revolving_limit','any_tax_liens']
other_cols = ['employment','home_ownership_status']
#categorical cols with mode
for i in cat_cols:
    mode = train_csv[i].mode()[0]
    train_csv[i].fillna(mode,inplace = True)
    train_csv[i].replace([np.inf,-np.inf],mode,inplace= True)
#numerical with median
for i in num_cols:
    median = train_csv[i].median()
    train_csv[i].fillna(median,inplace=True)
    train_csv[i].replace([np.inf,-np.inf],median,inplace=True)
for i in other_cols:
    unknown = "unknown"
    train_csv[i].fillna(unknown,inplace= True)
    train_csv[i].replace([np.inf,-np.inf],unknown,inplace=True)
    

In [58]:
#finding corr
correlation_mat = train_csv.corr()
#mat.figure(figsize = (20,20))
#sns.heatmap(correlation_mat,annot=True)
print(train_csv["requested_amnt"].corr(train_csv["loan_paid"]))
print(train_csv["annual_income"].corr(train_csv["loan_paid"]))
loan_income_ratio = train_csv["requested_amnt"] / train_csv["annual_income"]
print(loan_income_ratio.corr(train_csv["loan_paid"]))
train_csv["loan_income_ratio"] = loan_income_ratio
print(train_csv)
fico_mean = (train_csv["fico_score_range_low"]+train_csv['fico_score_range_high'])/2
fico_range = (train_csv['fico_score_range_high']-train_csv['fico_score_range_low'])
train_csv.drop(columns=["fico_score_range_high","fico_score_range_low"],inplace=True)
train_csv['fico_mean'] = fico_mean
train_csv["fico_range"] = fico_range
#print(train_csv)



-0.06587877771437163
0.04133225140212284
nan
        requested_amnt loan_duration                      employment  \
0              32000.0     60 months                             SVP   
1               6000.0     36 months    METAL FABRICATION SUPERVISOR   
2               4200.0     36 months  insurance collector specialist   
3               9725.0     36 months                 Kitchen Manager   
4              24000.0     36 months              Appeals Specialist   
...                ...           ...                             ...   
999995          8000.0     36 months         Human Resources Analyst   
999996          4000.0     36 months                 Support Manager   
999997         11000.0     60 months             software specialist   
999998         14400.0     36 months                             QMS   
999999         14000.0     60 months                 Unit Controller   

       employment_length     reason_for_loan  annual_income  \
0                4 years  d

In [59]:
loan_duration = []
employment_length = []
for i in range(train_csv.shape[0]):
    loan_duration.append(train_csv['loan_duration'].iloc[i].replace(" months",""))
    employment_length.append(train_csv['employment_length'].iloc[i].replace("< 1 year","0").replace(" years","").replace(" year","").replace("+",""))
train_csv.drop(columns=["loan_duration","employment_length"],inplace=True)
train_csv["loan_duration"] = loan_duration
train_csv["employment_length"] = employment_length
print(train_csv)

        requested_amnt                      employment     reason_for_loan  \
0              32000.0                             SVP  debt_consolidation   
1               6000.0    METAL FABRICATION SUPERVISOR    home_improvement   
2               4200.0  insurance collector specialist  debt_consolidation   
3               9725.0                 Kitchen Manager  debt_consolidation   
4              24000.0              Appeals Specialist  debt_consolidation   
...                ...                             ...                 ...   
999995          8000.0         Human Resources Analyst    home_improvement   
999996          4000.0                 Support Manager  debt_consolidation   
999997         11000.0             software specialist  debt_consolidation   
999998         14400.0                             QMS         credit_card   
999999         14000.0                 Unit Controller  debt_consolidation   

        annual_income  debt_to_income_ratio employment_verified